In [1]:
%load_ext autoreload
%autoreload 2

# System preparation

In [2]:
import molsysmt as msm

In [3]:
molsys = msm.convert("pdbid:1QKU", selection='chain_name==["B", "E"] and molecule_type=="small molecule"')

/home/diego/Projects/MolSysMT/molsysmt/native/io/topology/classes/mmtf_MMTFDecoder.py:43: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)


In [4]:
msm.view(molsys)

NGLWidget()

In [5]:
msm.info(molsys)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_small_molecules,n_frames
molsysmt.MolSys,20,1,1,1,1,1,1,1


In [6]:
msm.convert(molsys, 'ligand.pdb')

'ligand.pdb'

## With the Gaff forcefield

See: 
    
    https://github.com/openforcefield/openff-toolkit/issues/121
    https://github.com/ADicksonLab/mastic/blob/58749c40fe364110e3e7be8aa79a89f32d956d09/mastic/interfaces/rdkit.py
    https://github.com/tdudgeon/simple-simulate-complex/blob/master/simulateComplexWithSolvent.py
    https://github.com/openmm/openmmforcefields
    https://open-forcefield-toolkit.readthedocs.io/en/latest/examples.html


In [7]:
from openff.toolkit.topology import Molecule, Topology
from rdkit import Chem
from rdkit.Chem import AllChem

In [8]:
ligand = Chem.MolFromPDBFile('ligand.pdb')

In [9]:
smiles = 'C[C@]12CC[C@@H]3c4ccc(cc4CC[C@H]3[C@@H]1CC[C@@H]2O)O'
template = Chem.MolFromSmiles(smiles)

In [10]:
ligand = AllChem.AssignBondOrdersFromTemplate(template, ligand)

In [11]:
ligand = Chem.AddHs(ligand,addCoords=True)

In [12]:
ligand = Molecule.from_rdkit(ligand)

In [13]:
ligand

NGLWidget()

In [35]:
from simtk.openmm.app import Modeller, ForceField
from openmmforcefields.generators import GAFFTemplateGenerator
from simtk import unit
from simtk.openmm import Vec3
import numpy as np

In [19]:
gaff = GAFFTemplateGenerator(molecules=ligand)

In [20]:
forcefield = ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml', 'amber/tip3p_HFE_multivalent.xml')

In [21]:
forcefield.registerTemplateGenerator(gaff.generator)

In [37]:
topology = ligand.to_topology().to_openmm()
positions = ligand.conformers[0]
modeller = Modeller(topology, positions)

max_size = max(max((pos[i] for pos in positions))-min((pos[i] for pos in positions)) for i in range(3))
vectors = Vec3(1,0,0), Vec3(0,1,0), Vec3(0.5,0.5,np.sqrt(2)/2) # vectors for rhombic dodecahedron
box_vectors = [(max_size+14.0*unit.angstroms)*v for v in vectors]
modeller.addSolvent(forcefield, model='tip3p', boxVectors=box_vectors)

In [45]:
msm.info(modeller)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_small_molecules,n_frames
openmm.Modeller,920,293,293,2,293,2,292,1,1


In [41]:
molsys = msm.convert(modeller)

In [42]:
molsys = msm.wrap_to_mic(molsys, center_of_selection='molecule_type=="small molecule"')

In [44]:
msm.view(molsys, standardize=True, water_as_surface=True)

NGLWidget()

In [46]:
msm.convert(molsys, to_form='ligand_solvated.pdb')

'ligand_solvated.pdb'

Sources:
https://github.com/tdudgeon/simple-simulate-complex    


## Bibliography

[Koes, David Ryan. “Computer-Aided Drug Discovery,” 167–88. Methods in Pharmacology and Toxicology, 2015.](https://doi.org/10.1007/7653_2015_46)

In [ ]:
https://open-forcefield-toolkit.readthedocs.io/en/latest/examples.html